## Import libraries

In [1]:
import pymysql
from openpyxl import Workbook
from openpyxl import load_workbook
import numpy as np

## Connect to database using PyMySQL

In [2]:
conn = pymysql.connect(host='localhost', user='root', password='autoset', db='travel', charset='utf8')

In [3]:
cursor = conn.cursor(pymysql.cursors.DictCursor)

In [4]:
import pandas as pd

### 고객 테이블 가입 정보 입력받기

In [5]:
print('이름을 입력하세요.')
member_name = input()
print('나이대를 입력하세요.')
member_age = input()
print('ID를 입력하세요.')
member_ID = input()
print('비밀번호를 입력하세요.')
member_PW = input()

이름을 입력하세요.
김혜린
나이대를 입력하세요.
20
ID를 입력하세요.
heahea
비밀번호를 입력하세요.
0127


### 고객 가입 정보 DB에 추가

In [6]:
sql = "insert into 고객(고객이름,나이대,회원ID,회원PW) values(%s, %s, %s, %s)"
cursor.execute(sql, (member_name, member_age, member_ID, member_PW))
conn.commit()

df_member = pd.read_sql("select * from 고객", conn)
df_member

,고객번호,고객이름,나이대,회원ID,회원PW
0,14,김혜린,20,hea990,0127
1,15,ㅇㅇㅇ,20,asdf,asdf
2,16,문혜원,20,moon98,hyewon
3,17,김혜린,20,hea990,0127
4,18,김혜린,20,heahea,0127
5,19,김혜린,20,heahea,0127
6,20,김혜린,20,heahea,0127


### 여행지 추천을 위한 고객 취향 정보 입력받기

In [7]:
print('ID를 입력하세요.')
prefer_ID = input()
print('여행 가고 싶은 달을 입력하세요.')
prefer_month = input()
print('가고 싶은 관광지 유형을 입력하세요.')
prefer_tour = input()
print('먹고 싶은 음식을 입력하세요.')
prefer_food = input()
print('선호하는 축제 유형을 입력하세요.')
prefer_festival = input()

ID를 입력하세요.
heahea
여행 가고 싶은 달을 입력하세요.
7
가고 싶은 관광지 유형을 입력하세요.
고택
먹고 싶은 음식을 입력하세요.
짬뽕
선호하는 축제 유형을 입력하세요.
먹거리


### 점수 기록 dataframe

In [8]:
df_score = pd.read_sql("select * from 지역",conn)
df_score['관광지개수'] = 0
df_score['축제개수'] = 0
df_score['음식점개수'] = 0
df_score['총점'] = 0

## 관광지 데이터 전처리

In [9]:
df_spot = pd.read_sql("select * from 관광지", conn)
df_spot

,관광지번호,관광지명,전화번호,소분류,지역코드,주소
0,L0001,가평 아홉마지기마을,031-582-3115,농.산.어촌 체험,I28,경기 가평군 가평읍 용추로 238
1,L0002,가평 이화원 나비스토리,031-581-0228,테마공원,I28,경기도 가평군 가평읍 자라섬로 64
2,L0003,가평 현충탑,031-5802-0667,기념탑/전망대,I28,경기도 가평군 가평읍 호반로 2540 가평읍 부근
3,L0004,가평군목공예영농조합,031-581-0550,전시관,I28,경기도 가평군 가평읍 경춘로 2045-65 가평읍
4,L0005,가평의병3.1항일운동기념비,031-580-2066,기념탑/전망대,I28,경기 가평군 가평읍 대곡리 467번지
...,...,...,...,...,...,...
8042,L8049,운암정,033-340-2224,유적지,J09,강원도 횡성군 횡성읍 한우로 191 횡성읍
8043,L8050,종가집 횡성공장,NULL,식음료,J09,강원도 횡성군 횡성읍 청용길 15
8044,L8051,횡성 밤두둑마을,033-344-7675,농.산.어촌 체험,J09,강원도 횡성군 횡성읍 반곡3길 6 횡성읍
8045,L8052,횡성향교,033-340-2224,유적지,J09,강원도 횡성군 횡성읍 향교로 12 횡성읍


In [10]:
onehot_spot = pd.get_dummies(df_spot.loc[:,['관광지번호', '지역코드', '소분류']],columns= ['소분류']) 
onehot_spot

,관광지번호,지역코드,소분류_강,소분류_계곡,소분류_고궁,소분류_고택,소분류_공단,소분류_공연장,소분류_공원,소분류_관광농원,...,소분류_터널,소분류_테마공원,소분류_폭포,소분류_항구/포구,소분류_해수욕장,소분류_해안절경,소분류_헬스투어,소분류_호수,소분류_화학-금속,소분류_희귀동.식물
0,L0001,I28,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,L0002,I28,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,L0003,I28,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,L0004,I28,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,L0005,I28,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8042,L8049,J09,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8043,L8050,J09,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8044,L8051,J09,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8045,L8052,J09,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 관광지 취향에 맞춰 점수 내기

In [11]:
category1 = 'null'
for c_name in list(onehot_spot):
    if prefer_tour in c_name:
        category1 = c_name

In [12]:
plus = 0
for loc_num in range(len(df_score['지역코드'])):    # 지역코드 개수로 돌림
    for spot_num in range(len(onehot_spot)):        # 관광지 데이터 개수로 돌림
        if df_score['지역코드'][loc_num] == onehot_spot['지역코드'][spot_num]:    # 결과df랑 관광지 지역이랑 같은 지역이면
            plus += onehot_spot[category1][spot_num]
    df_score['관광지개수'][loc_num] = plus
    plus = 0

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## 축제 테이블 전처리

In [13]:
df_festival = pd.read_sql("select * from 축제",conn)

In [14]:
onehot_festival = pd.get_dummies(df_festival.loc[:,['축제번호','지역코드','축제기간','소분류']],columns = ['소분류'])

### 축제 취향에 맞춰 점수 내기

In [15]:
category2 = 'null'

for c_name in list(onehot_festival):
    if prefer_festival in c_name:
        category2 = c_name

In [16]:
plus = 0
for loc_num in range(len(df_score['지역코드'])): # 지역코드갯수로 돌림
    for fest_num in range(len(onehot_festival)): # 축제데이터 갯수로 돌림
        if df_score['지역코드'][loc_num] == onehot_festival['지역코드'][fest_num]: # 결과df랑 축제 지역이랑 같은 지역이면
            if prefer_month == onehot_festival['축제기간'][fest_num]: # 여행예정달이랑 축제 기간이 같으면
                plus += onehot_festival[category2][fest_num]
    df_score['축제개수'][loc_num] = plus
    plus = 0

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [17]:
pd.set_option('display.max_rows', 200)

## 숙소 테이블 전처리

In [18]:
df_sleep = pd.read_sql("select * from 숙소", conn)

In [19]:
onehot_sleep = pd.get_dummies(df_sleep.loc[:,['숙소번호','업종명']],columns= ['업종명']) 

## 음식점 데이터 전처리

In [20]:
df_food = pd.read_sql("select * from 음식점",conn)

### 음식 취향에 맞춰 점수 내기

In [21]:
columns = 0

for fdc in df_food['설명']: # df_food 설명에 있는걸 한개씩 가져옴
    if prefer_food in fdc:
        for loc_code in range(len(df_score['지역코드'])): # 지역코드 갯수 만큼 for문 돌려라
            if df_score['지역코드'][loc_code] == df_food['지역코드'][columns]: 
                # df_score의 지역코드와 df_food의 지역코드가 같으면 음식점 갯수에 1 더하기
                df_score['음식점개수'][loc_code]+=1
        #print(df_food['음식점번호'][columns])
    columns+=1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [22]:
df_score

,지역코드,지역명,위도,경도,관광지개수,축제개수,음식점개수,총점
0,A01,서울특별시,37.563569,126.980008,9,0,11,0
1,B01,부산광역시,35.177019,129.076953,0,0,3,0
2,C01,대구광역시,35.868542,128.603553,5,2,1,0
3,D01,인천광역시,37.453233,126.707353,3,0,4,0
4,E01,광주광역시,35.156975,126.853364,5,1,2,0
5,F01,대전광역시,36.347119,127.386567,0,0,4,0
6,G01,울산광역시,35.535408,129.313689,0,0,1,0
7,H01,세종특별자치시,36.480012,127.289069,0,0,0,0
8,I01,경기도 수원시,37.267556,127.023389,0,0,1,0
9,I02,경기도 고양시,37.672486,126.752778,0,0,1,0


### 최종 점수 내기

In [23]:
df_result = pd.DataFrame(columns=['지역명', '총점'])

In [24]:
## 그냥 조건 없이 다 더해서 총점
plus = 0
for loc_num in range(len(df_score)):
    plus = df_score['관광지개수'][loc_num] + df_score['축제개수'][loc_num] + df_score['음식점개수'][loc_num]
    df_score['총점'][loc_num] = plus
    plus = 0

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [25]:
## 모든 취향이 1 이상인 지역만 df_result로 넣기
res_num = 0
for loc_num in range(len(df_score)):
    if df_score['관광지개수'][loc_num] > 0 and df_score['축제개수'][loc_num] > 0 and df_score['음식점개수'][loc_num] > 0 :
        df_result.loc[res_num,'지역명'] = df_score['지역명'][loc_num]
        df_result.loc[res_num,'총점'] = df_score['총점'][loc_num]
        res_num+=1
        
df_result

,지역명,총점
0,대구광역시,8
1,광주광역시,8
2,전라북도 임실군,4
3,제주도,10


In [26]:
## 모든 취향이 1 이상인 지역이 없다면 취향 셋 중에 두개가 1 이상인 것을 df_result에 넣기
a = 0
b = 0
c = 0
if df_result.empty:
    for loc_num in range(len(df_score)):
        if df_score['관광지개수'][loc_num] > 0 :
            a = 1
        if df_score['축제개수'][loc_num] > 0 :
            b = 1
        if df_score['음식점개수'][loc_num] > 0 :
            c = 1
        if a+b+c > 1 :
            df_result.loc[res_num,'지역명'] = df_score['지역명'][loc_num]
            df_result.loc[res_num,'총점'] = df_score['총점'][loc_num]
        a=0
        b=0
        c=0

### 순서대로 정렬         
df_sorted_result = df_result.sort_values(by='총점', ascending=False)
df_sorted_result

,지역명,총점
3,제주도,10
0,대구광역시,8
1,광주광역시,8
2,전라북도 임실군,4


### 추천 결과 중 가고 싶은 지역을 선택

In [27]:
print('추천된 결과 중 여행을 가고 싶은 지역의 번호를 고르세요.')
picknum = input()
place = df_sorted_result['지역명'][int(picknum)]

placecode = 'null'
for count in range(len(df_score)):
    if place == df_score['지역명'][count]:
        placecode = df_score['지역코드'][count]
        break

추천된 결과 중 여행을 가고 싶은 지역의 번호를 고르세요.
3


### 검색기록 DB에 추가

In [28]:
memnum = 0
for loc_num in range(len(df_member)): # 고객 명수로 돌림
    if prefer_ID == df_member['회원ID'][loc_num]:
        memnum = df_member['고객번호'][loc_num]
        break

In [29]:
sql = "insert into 검색기록(고객번호,여행예정월,관광지취향,음식점취향,축제취향,추천결과) values(%s, %s, %s, %s, %s, %s)"
cursor.execute(sql, (str(memnum), prefer_month, prefer_tour, prefer_food, prefer_festival, placecode))
conn.commit()

In [30]:
df_search = pd.read_sql("select * from 검색기록", conn)

In [49]:
df_search

,검색기록번호,고객번호,여행예정월,관광지취향,음식점취향,축제취향,추천결과
0,13,16,7,고택,짬뽕,먹거리,Q01
1,14,18,7,고택,짬뽕,먹거리,Q01
2,15,18,7,고택,짬뽕,먹거리,Q01


### 고객이 선택한 지역의 취향에 맞는 관광지 보여주기

In [31]:
df_resultspot = pd.DataFrame(columns=['관광지명','위도','경도'])
df_resultfood = pd.DataFrame(columns=['음식점명','위도','경도'])
df_resultfes = pd.DataFrame(columns=['축제명','위도','경도'])

In [50]:
for loc_num in range(len(df_spot)):
    if placecode == df_spot['지역코드'][loc_num]:
        if prefer_tour == df_spot['소분류'][loc_num]:
             df_resultspot.loc[loc_num,'관광지명'] = df_spot['관광지명'][loc_num]
df_resultspot

,관광지명,위도,경도
3189,양금석가옥,33.2913285653559,126.62848088838
3200,서귀포 김정희 유배지,33.2504056192256,126.278196318407
3276,이중섭 거주지,33.2455483773808,126.564327488975
3293,성읍고평오가옥,33.3858661567386,126.801686030396
3295,이영숙가옥,33.3872412249184,126.800062153468


### 고객이 선택한 지역의 취향에 맞는 축제 보여주기

In [33]:
for loc_num in range(len(df_festival)):
    if placecode == df_festival['지역코드'][loc_num]:
        if prefer_festival == df_festival['소분류'][loc_num]:
            if prefer_month == df_festival['축제기간'][loc_num]:
                df_resultfes.loc[loc_num,'축제명'] = df_festival['축제명'][loc_num]

### 고객이 선택한 지역의 취향에 맞는 음식점 보여주기

In [34]:
for i in df_food.index:
    if placecode == df_food['지역코드'][i]:
        if prefer_food in df_food['설명'][i]:
            df_resultfood.loc[i,'음식점명'] = df_food['음식점명'][i]

### 음식점 위도 경도 가져오기 (kakao api 사용)

In [35]:
import requests

for i in df_resultfood.index:
    searching = df_resultfood['음식점명'][i]
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(searching)
    headers = {"Authorization": "KakaoAK 3b208964b53cdafc6f57db17e800d418"}
    places = requests.get(url, headers = headers).json()['documents']
    df_resultfood['경도'][i] = places[0]['x']
    df_resultfood['위도'][i] = places[0]['y']
    
df_resultfood

,음식점명,위도,경도
5541,금문도,37.5278459019907,126.72577368819
5555,다래향,33.5458153322043,126.660687856748


### 축제 위도 경도 가져오기

In [36]:
import matplotlib.pyplot as plt
%matplotlib inline

In [37]:
df_festwhere = pd.read_excel('C:/Users/건욱/Desktop/김혜린/3학년/엔지니어링DB 및 실습/플젝/데이터/축제 위도경도 포함.xls')

In [38]:
for i in df_resultfes.index:
    for count in range(len(df_festwhere)):
        if df_resultfes['축제명'][i] == df_festwhere['축제명'][count]:
            df_resultfes['위도'][i] = df_festwhere['위도'][count]
            df_resultfes['경도'][i] = df_festwhere['경도'][count]

df_resultfes

,축제명,위도,경도
918,금능 원담축제,33.3911,126.236
937,이호테우축제,33.4962,126.456
945,제7회 월대천 축제,33.4923,126.435


### 관광지 위도 경도 가져오기

In [39]:
for i in df_resultspot.index:
    searching = df_resultspot['관광지명'][i]
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(searching)
    headers = {"Authorization": "KakaoAK 3b208964b53cdafc6f57db17e800d418"}
    places = requests.get(url, headers = headers).json()['documents']
    df_resultspot['경도'][i] = places[0]['x']
    df_resultspot['위도'][i] = places[0]['y']
    
df_resultspot

,관광지명,위도,경도
3189,양금석가옥,33.2913285653559,126.62848088838
3200,서귀포 김정희 유배지,33.2504056192256,126.278196318407
3276,이중섭 거주지,33.2455483773808,126.564327488975
3293,성읍고평오가옥,33.3858661567386,126.801686030396
3295,이영숙가옥,33.3872412249184,126.800062153468


### 지도에 위치 찍기 > 시각화

In [40]:
import folium
from pandas import DataFrame
from pandas import ExcelFile

In [41]:
df_local = pd.read_sql("select * from 지역", conn)

### 관광지 지도에 찍기

In [42]:
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

In [43]:
wi = 0        # 선택한 지역 위도 경도 받아오기
gyung = 0
for i in df_local.index:
    if df_local['지역코드'][i] == placecode:
        wi = df_local['위도'][i]
        gyung = df_local['경도'][i]
        break

In [44]:
map_try = folium.Map(location=[wi,gyung], zoom_start=12)

In [45]:
# 관광지 찍기
for i in df_resultspot.index:
    # 행 우선 접근 방식으로 값 추출하기
    name = df_resultspot.loc[i, '관광지명']
    lat = df_resultspot.loc[i, '위도']
    lng = df_resultspot.loc[i, '경도']
    
    # 추출한 정보를 지도에 표시
    marker1 = folium.Marker([lat,lng], popup=name, icon=folium.Icon(color='blue'))
    marker1.add_to(map_try)

In [46]:
# 축제 찍기
for i in df_resultfes.index:
    # 행 우선 접근 방식으로 값 추출하기
    name = df_resultfes.loc[i, '축제명']
    lat = df_resultfes.loc[i, '위도']
    lng = df_resultfes.loc[i, '경도']
    
    # 추출한 정보를 지도에 표시
    marker2 = folium.Marker([lat,lng], popup=name, icon=folium.Icon(color='red'))
    marker2.add_to(map_try)

In [1]:
# 음식점 찍기
for i in df_resultfood.index:
    name = df_resultfood.loc[i, '음식점명']
    lat = df_resultfood.loc[i, '위도']
    lng = df_resultfood.loc[i, '경도']
    marker3 = folium.Marker([lat,lng], popup=name, icon=folium.Icon(color='green'))
    marker3.add_to(map_try)

map_try.save('try.html') ## html 파일로 열면 한글 안 깨짐
map_try  

NameError: name 'df_resultfood' is not defined

In [48]:
conn.close()